# Connect to the real YoMap Neo4j DB

https://yomap-neo-dev.fly.dev/browser/

In [1]:
from neo4j import GraphDatabase

In [2]:
NEO4J_URI = "neo4j+s://yomap-neo-dev.fly.dev:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "5GTAmrydyizxgRcY"

In [3]:
# Connect to the Neo4j database
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [23]:
path_query = """
    MATCH (user:Profile {display_name: $profile_name})
    MATCH (category:Category {name: $category_name})<-[:PROVIDER_OF]-(provider:Profile)

    WHERE point.distance(provider.location, user.location) / 1000 < 10
    MATCH path = shortestPath((user)-[*]-(provider))
    WHERE length(path) >= 1

    WITH provider, category, user, path,
        point.distance(provider.location, user.location) / 1000 AS distance_km,
        length(path) AS path_length

    RETURN 
        provider.display_name AS provider_name, 
        category.name AS service,
        distance_km,
        path_length,
        path
    ORDER BY distance_km ASC, path_length ASC
    LIMIT 5;
"""

In [25]:
with driver.session() as session:
        result = session.run(
            path_query,
            profile_name="Noel 2",
            category_name="pets"
        )

        # Print results
        for record in result:
            print(f"Provider: {record['provider_name']} | Distance: {record['distance_km']:.2f} km | Path: {record['path_length']:.0f}")

Provider: Violeta IOS | Distance: 1.72 km | Path: 2
Provider: violeta ios 0.7.17 | Distance: 1.94 km | Path: 2
Provider: Andres Laguna | Distance: 2.70 km | Path: 2
Provider: Higal Ramal | Distance: 7.50 km | Path: 2


In [ ]:
driver.execute_query("MATCH (n:Profile) RETURN n.displayName")

EagerResult(records=[<Record n=<Node element_id='4:bb30cff9-b771-442c-b900-273061cbd871:0' labels=frozenset({'Profile'}) properties={'gender': '', 'service': 'user', 'displayName': 'Arizza', 'profileId': '32z0evf2wJBkKNMaZYql', 'photo': '', 'location': POINT(-79.55968186016943 8.994300842285156), 'userId': 'DGnrQapGUTPhGJ6Noq2GuuDCzM82'}>>, <Record n=<Node element_id='4:bb30cff9-b771-442c-b900-273061cbd871:1' labels=frozenset({'Profile'}) properties={'gender': 'female', 'service': 'user', 'displayName': 'Oriana Vega24422', 'profileId': '3QWnkBxpnLYq4bWrLrx0', 'photo': 'Files/Users/scXuutTfVsgox7LShTB9i1vShaM2/file_1701890567783-407746_Screenshot_2023-11-20-10-53-55-423_com.facebook.katana.jpg', 'location': POINT(-70.2067349 11.7080935), 'userId': 'scXuutTfVsgox7LShTB9i1vShaM2', 'age': 27}>>, <Record n=<Node element_id='4:bb30cff9-b771-442c-b900-273061cbd871:2' labels=frozenset({'Profile'}) properties={'gender': 'male', 'service': 'misc', 'displayName': 'Alejandro ', 'profileId': '4TJOz

In [41]:
with driver.session() as session:
        result = session.run("MATCH (n:Profile) RETURN n.displayName as name")

        for record in result:
                print(record['name'])

Arizza
Oriana Vega24422
Alejandro 
Jules D.
Daniela Febles
Akshdeep
Harry
Manjeet kaur
Jino 
Ricardo De Leon
shankar
Noel Xiaomi
joginder singh
YoMap Assistant
Mario Redmi 10A
Anita 
ali
Fito Viroka
Harpreet Singh
Oges Fbios
vani manda
sachin kharre
Arka
abhi
Jean Sams 7.16
Rutiner Dasheeri
Readet Ferche
Ricky Barranquilla 
lucy
manpreet
Alex
YoMap LG
Navratan Singh 
Test user 63636363
Pulor Baszo
ummed Ali 
YM Soporte stage
Harsh Tiwari 
Kuldeep Singh 
laurent
Ewa 
Amit Kumar
Costura Ana Mer bis
Carolina 7.17
Luis Ortiz
ice panama
Andres Laguna
Alex
Higal Ramal
Redmi 10A
machhitrupti17@gmail.com
Ciloo 
Gabriel
Noel 2
Alexandre Rio
Test Real Number Colombia
Гульназира 
Millycen Hurtado
Nasir Ahmad Dar
Tarandeep singh 
Angela 
Laissa
Janicy dos Santos 
Violeta IOS
HomeSecurePTY
Emulator 66980917
violeta ios 0.7.17
Shiro
Jule Yune
Leone Isog
Noel Moreno
deleter2
new jean sams atage 716
JM sams2 stage 7.16
Mario Test
Mario
Mario S
Mario Test Service
Mario Emulator 46
Mario Test Hide 46
je

In [51]:
with driver.session() as session:
    result = session.run("MATCH p=()-[r:CONNECTS]->() RETURN p")

    for record in result:
        path = record["p"]  # Extract the path

        # Extract start and end nodes
        start_node = path.start_node
        end_node = path.end_node

        # Extract properties from both nodes
        start_profile = {
            "displayName": start_node["displayName"],
            "profileId": start_node["profileId"],
            "service": start_node["service"],
            "photo": start_node["photo"],
            "location": start_node["location"],  # This is a POINT
            "age": start_node["age"],
            "gender": start_node["gender"],
            "userId": start_node["userId"]
        }

        end_profile = {
            "displayName": end_node["displayName"],
            "profileId": end_node["profileId"],
            "service": end_node["service"],
            "photo": end_node["photo"],
            "location": end_node["location"],
            "age": end_node.get("age", "N/A"),  # Handle missing age
            "gender": end_node.get("gender", "N/A"),
            "userId": end_node["userId"]
        }

        print(f"🔗 {start_profile['displayName']} ({start_profile['service']}) ➝ {end_profile['displayName']} ({end_profile['service']})")
        print(f"   📍 {start_profile['location']} ➝ {end_profile['location']}")
        print(f"   🆔 {start_profile['profileId']} ➝ {end_profile['profileId']}\n")

# Close the connection
driver.close()

🔗 Fito Viroka (repair) ➝ Laissa (transport)
   📍 POINT(-79.5204350214493 8.975854419174278) ➝ POINT(-79.5326233 8.9767517)
   🆔 K9OxGBqG3q2iDpjvWaSo ➝ v34DujEeoB8wx4zysUQr

🔗 Laissa (transport) ➝ Fito Viroka (repair)
   📍 POINT(-79.5326233 8.9767517) ➝ POINT(-79.5204350214493 8.975854419174278)
   🆔 v34DujEeoB8wx4zysUQr ➝ K9OxGBqG3q2iDpjvWaSo

🔗 Costura Ana Mer bis (misc) ➝ Readet Ferche (spa)
   📍 POINT(-79.3912687 9.0622907) ➝ POINT(-79.28913876414299 9.086820241357247)
   🆔 eHD8pCRigDXreCk8VkVI ➝ ceqZenAYRFFNZgXrp2Ts

🔗 Readet Ferche (spa) ➝ Costura Ana Mer bis (misc)
   📍 POINT(-79.28913876414299 9.086820241357247) ➝ POINT(-79.3912687 9.0622907)
   🆔 ceqZenAYRFFNZgXrp2Ts ➝ eHD8pCRigDXreCk8VkVI

🔗 Costura Ana Mer bis (misc) ➝ Jean Sams 7.16 (pets)
   📍 POINT(-79.3912687 9.0622907) ➝ POINT(-79.5273606 8.978922)
   🆔 eHD8pCRigDXreCk8VkVI ➝ T8slg6VM8mJDss4G6sr7

🔗 Jean Sams 7.16 (pets) ➝ Costura Ana Mer bis (misc)
   📍 POINT(-79.5273606 8.978922) ➝ POINT(-79.3912687 9.0622907)
   🆔 T8s

In [53]:
import pandas as pd
from neo4j import GraphDatabase

NEO4J_URI = "neo4j+s://yomap-neo-dev.fly.dev:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "5GTAmrydyizxgRcY"

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def get_profile_connections():
    data = []  # List to store extracted connections

    with driver.session() as session:
        result = session.run("MATCH p=()-[r:CONNECTS]->() RETURN p")

        for record in result:
            path = record["p"]  # Extract the path

            # Extract start and end nodes
            start_node = path.start_node
            end_node = path.end_node

            # Extract properties from both nodes
            start_profile = {
                "displayName_start": start_node["displayName"],
                "profileId_start": start_node["profileId"],
                "service_start": start_node["service"],
                "photo_start": start_node["photo"],
                "location_start": start_node["location"],  # POINT type
                "age_start": start_node.get("age", "N/A"),
                "gender_start": start_node.get("gender", "N/A"),
                "userId_start": start_node["userId"]
            }

            end_profile = {
                "displayName_end": end_node["displayName"],
                "profileId_end": end_node["profileId"],
                "service_end": end_node["service"],
                "photo_end": end_node["photo"],
                "location_end": end_node["location"],
                "age_end": end_node.get("age", "N/A"),
                "gender_end": end_node.get("gender", "N/A"),
                "userId_end": end_node["userId"]
            }

            # Merge start & end profiles into a single dictionary
            connection = {**start_profile, **end_profile}
            data.append(connection)  # Append to list

    # Convert list to DataFrame
    df = pd.DataFrame(data)
    return df

# Fetch connections & store in DataFrame
df_connections = get_profile_connections()

# Close the Neo4j connection
driver.close()




In [55]:
df_connections.to_csv("df_connections.csv")

In [ ]:
from neo4j import GraphDatabase

# Neo4j connection details
NEO4J_URI = "bolt://your-remote-server-ip:7687"
USERNAME = "neo4j"
PASSWORD = "your_password"

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(USERNAME, PASSWORD))

def get_profile_connections():
    with driver.session() as session:
        result = session.run("MATCH p=()-[r:CONNECTS]->() RETURN p")

        for record in result:
            path = record["p"]  # Extract the path

            # Extract start and end nodes
            start_node = path.start_node
            end_node = path.end_node

            # Extract properties from both nodes
            start_profile = {
                "displayName": start_node["displayName"],
                "profileId": start_node["profileId"],
                "service": start_node["service"],
                "photo": start_node["photo"],
                "location": start_node["location"],  # This is a POINT
                "age": start_node["age"],
                "gender": start_node["gender"],
                "userId": start_node["userId"]
            }

            end_profile = {
                "displayName": end_node["displayName"],
                "profileId": end_node["profileId"],
                "service": end_node["service"],
                "photo": end_node["photo"],
                "location": end_node["location"],
                "age": end_node.get("age", "N/A"),  # Handle missing age
                "gender": end_node.get("gender", "N/A"),
                "userId": end_node["userId"]
            }

            print(f"🔗 {start_profile['displayName']} ({start_profile['service']}) ➝ {end_profile['displayName']} ({end_profile['service']})")
            print(f"   📍 {start_profile['location']} ➝ {end_profile['location']}")
            print(f"   🆔 {start_profile['profileId']} ➝ {end_profile['profileId']}\n")

# Run the function
get_profile_connections()

# Close the connection
driver.close()


In [4]:
from yfiles_jupyter_graphs import GraphWidget

In [12]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "cassandra"

In [28]:
query_1 = """
    MATCH (p:Profile)-[r1:PROVIDER_OF]->(c:Category)
    OPTIONAL MATCH (p)-[r2:CONNECTS]-(p2:Profile)
    RETURN p, r2
"""

query_2 = """
    MATCH (p:Profile)-[r2:CONNECTS]-(p2:Profile) RETURN p, r2
"""

In [29]:


def show_graph():
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    session = driver.session()

    widget = GraphWidget(graph=session.run(query_2).graph())

    widget.node_label_mapping = "id"

    return widget

show_graph()

GraphWidget(layout=Layout(height='740px', width='100%'))